In [1]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
reviews = []
ratings = []

for i in range(0,600,20):
    url2 = 'https://www.yelp.com/biz/kims-sushi-west-orange?osq=sushi' + str(i)
    response = requests.get(url2, headers=headers, verify=False).text
    soup = BeautifulSoup(response, "html.parser")
    time.sleep(5)
    rblock= soup.find_all('div', attrs={'lemon--div__373c0__1mboc arrange-unit__373c0__1piwO arrange-unit-grid-column--8__373c0__2yTAx border-color--default__373c0__2oFDT'})[1:421]
    
    for block in rblock:
        reviews.append(block.find('span',attrs={'lang':'en'}))
        ratings.append(block.find('div',class_=re.compile(r"^i-stars--regular")))
        
print(len(reviews))
print(len(ratings))

500
500


In [3]:
import gensim
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
rate=[]

cleanr = re.compile('<[^<]+?')
review = [re.sub(cleanr, '',a.text) for a in reviews]
for r in ratings:
    rate.append(r['aria-label'])

rating= [i.split(' star rating')[0] for i in rate]

In [5]:
sushi= pd.DataFrame({'reviews':review, 'ratings':rating})[0:500]

In [6]:
sushi.to_csv('sushi.csv')

In [37]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

from nltk.corpus import movie_reviews
import collections
from nltk.classify import NaiveBayesClassifier
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from imblearn.over_sampling import SMOTE
from nltk.tokenize import WordPunctTokenizer

import random

In [49]:
reviews = []
for row in sushi['reviews']:
    
    tokenizer = WordPunctTokenizer()
    words = tokenizer.tokenize(str(row))
    
    #removing punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    
    #removing stop words
    english_stops = set(stopwords.words('english'))
    characters_to_remove = ['...', '', '’', "n't", "'s'",
                            '”', "", ' ', '  ','..','.',str("'s"),".i",
                            '“','``']   
   
    #tokenizing words
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
    
    
    
    #Lematizing words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    reviews.append(lemma_list)

In [58]:
tokenized_reviews = []

for lr in reviews:
    tokenized_reviews.append(lr)


In [60]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(tokenized_reviews, size=100, window=5, min_count=1, workers=4)

In [62]:
w2v_model.wv.most_similar(positive=['sushi'], topn=5)

[('alot', 0.9992790222167969),
 ('state', 0.9992648363113403),
 ('thing', 0.999247670173645),
 ('year', 0.999241292476654),
 ('restaurant', 0.9992409944534302)]

In [95]:
w2v_model.wv.most_similar(positive=['crab'], topn=5)

[('eel', 0.9990442395210266),
 ('shrimp', 0.9989959597587585),
 ('tempura', 0.9989181756973267),
 ('baked', 0.9988822937011719),
 ('tried', 0.998764157295227)]

In [71]:
w2v_model.wv.most_similar(positive=['shrimp'], topn=5)

[('crab', 0.9989959597587585),
 ('eel', 0.998913049697876),
 ('ball', 0.9986932873725891),
 ('roll', 0.9984256029129028),
 ('cucumber', 0.9983853697776794)]

In [73]:
w2v_model.save('hot_topic_1')

From the word2vec model, we can infer that the words associated with the words chosen make sense as the sushi at SushiKim is "state"-renowned and you can get "a lot" of sushi, and so on. All the words that the model outlies are specific enough to the words searched through the database of tokenized words. 